In [79]:
import arff
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from operator import attrgetter, itemgetter
from io import StringIO
from sklearn.model_selection import train_test_split
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, confusion_matrix,classification_report,precision_recall_fscore_support as score, average_precision_score
from sklearn import metrics
from sklearn.model_selection import cross_validate

## training model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [80]:
data = arff.load(open('../text-blob-pt/OffComBR3.arff'))
df = pd.DataFrame(data['data'])
df.columns = ['hate', 'sentence']

# transforming 'yes' into 1 and 'no' into 0
df['hate'] = df['hate'].apply(lambda x: 1 if x == 'yes' else 0)

X = df['sentence'].tolist()
y = df['hate'].tolist()
X_train, X_test, y_train, y_test = train_test_split(
                                    X, y, test_size=0.33, random_state=42)

In [81]:
RandomForest =  Pipeline([
        ('tfidf',TfidfVectorizer( ngram_range=(1,4))),
        ('clf', RandomForestClassifier(n_estimators=100, max_depth=None, min_samples_leaf=1, min_samples_split=2, min_weight_fraction_leaf=0,)),
        ])

In [82]:
RandomForest.fit(X_train, y_train)
pred = RandomForest.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.82      0.93      0.87       267
           1       0.50      0.27      0.35        74

   micro avg       0.78      0.78      0.78       341
   macro avg       0.66      0.60      0.61       341
weighted avg       0.75      0.78      0.76       341



In [83]:
MLP = Pipeline([
        ('tfidf',TfidfVectorizer(ngram_range=(1, 1))),
        ('clf', MLPClassifier(activation='logistic', alpha=0, solver='lbfgs')),
        ])

In [84]:
MLP.fit(X_train, y_train)
pred = MLP.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.86      0.89      0.87       267
           1       0.54      0.47      0.50        74

   micro avg       0.80      0.80      0.80       341
   macro avg       0.70      0.68      0.69       341
weighted avg       0.79      0.80      0.79       341



In [85]:
SVCl = Pipeline([
        ('tfidf',TfidfVectorizer(ngram_range=(1, 1))),
        ('clf', SVC(C=4, kernel='linear', probability=True, shrinking=True, tol=1)),
        ])

In [86]:
SVCl.fit(X_train, y_train)
pred = SVCl.predict(X_test)
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.84      0.94      0.89       267
           1       0.62      0.34      0.44        74

   micro avg       0.81      0.81      0.81       341
   macro avg       0.73      0.64      0.66       341
weighted avg       0.79      0.81      0.79       341



In [138]:
voting = VotingClassifier(estimators=[('rf', RandomForest), ('mlp', MLP), ('svc', SVCl)], 
                          voting='soft', flatten_transform=False,
                          weights=[0.2, 2, 0.5], 
                          n_jobs=15)

In [ ]:
voting.fit(X_train, y_train)
pred = voting.predict(X_test)
print(classification_report(y_test, pred))